#### Es necesario aplicar el siguiente parche tan solo una vez a one_hot_encode para que no salga un error cada vez que ejecutemos las evaluaciones. Si no se aplica el siguiente código las evaluaciones darán el siguiente error por no haber muestras de distintas clasificaciones: 
---------------------------------------------------------------------------
KeyError                                  Traceback (most recent call last)
Cell In[1], line 59
     56 separated += separated_extra  # Inyectamos muestras de relleno
     58 # Paso 5: One-hot encoding
---> 59 one_hotted = data.one_hot_encode(separated)
     61 # Paso 6: Eliminar muestras de relleno
     62 one_hotted = one_hotted[:-len(extra_samples)]

File c:\Users\franm\TFG-XAI-CONCURRENCIA\Notebooks-Modificados\generate_data.py:117, in Data.one_hot_encode(self, samples_char_sep)
    114 # unique_chars.remove(True)
    115 # unique_chars.remove(False)
    116  unique_chars.remove('A')
--> 117  unique_chars.remove('R')
    118  unique_chars.remove('D')
    119  unique_chars.remove('V')

KeyError: 'R'

In [2]:
from generate_data import Data
import pandas as pd
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
import numpy as np

def safe_one_hot_encode(self, samples_char_sep):
    df = pd.DataFrame(samples_char_sep, columns=self.feature_names + ['Correct'])
    unique_chars = set()

    for col in df.columns[:-1]:  # Excluye la columna de la clase
        unique_chars = unique_chars.union(df[col].unique())

    # Evita errores por KeyError si no están presentes
    for label in ['A', 'R', 'D', 'V']:
        unique_chars.discard(label)

    unique_chars = sorted(list(unique_chars))

    self.le = LabelEncoder()
    self.le.fit(unique_chars)

    self.feature_names_one_hotted = []
    for i in range(3):
        for j in range(self.layer_size):
            for c in self.le.classes_:
                self.feature_names_one_hotted.append(f'f{i + 1}-{j}-{c}')

    df_no_label = df.drop(['Correct'], axis=1)
    df_encoded = df_no_label.apply(self.le.transform)

    unique_labels = set()
    for col in df_encoded:
        unique_labels = unique_labels.union(df_encoded[col].unique())
    df_unique_labels = pd.DataFrame(unique_labels)

    for i in range(1, 3 * self.layer_size):
        df_unique_labels[i] = df_unique_labels[0]
    self.unique_labels = df_unique_labels

    self.ohe = OneHotEncoder(categories='auto')
    self.ohe.fit(df_unique_labels)

    one_hotted_df = pd.DataFrame(self.ohe.transform(df_encoded).toarray(), columns=self.feature_names_one_hotted)
    self.num_one_hot_encodings = int(one_hotted_df.shape[1] / self.layer_size / 3)
    one_hotted_df['label'] = df['Correct']

    columns_one_hotted = list(one_hotted_df.columns.values[:-1])

    self.f1_start = 0
    self.f2_start = self.f1_start
    while (columns_one_hotted[self.f2_start].find('f2') == -1):
        self.f2_start += 1

    self.f3_start = self.f2_start
    while (columns_one_hotted[self.f3_start].find('f3') == -1):
        self.f3_start += 1

    self.f3_end = len(columns_one_hotted)

    return np.array(one_hotted_df)

# Aplicar el parche
Data.one_hot_encode = safe_one_hot_encode


#### Generamos el nuevo dataset y lo evaluamos para el modelo LSTM del 4º filtro

In [ ]:
import torch
import random
import pandas as pd
from generate_data import Data
from evaluate import get_wrong_predictions_bycases, print_wrong_preds_bycases
from models import LSTM_Model
from sklearn.preprocessing import LabelEncoder

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Paso 1: Inicializar generador de datos
data = Data(layer_size=15, interop_distances=[0], permutation_intervals=1)

# Paso 2: Generar muestras con f2='wu' y f3='rd'
def generate_atomicity_samples_rd(n=10):
    layer_size = 15
    base_f1 = ',.,_,,_,_,._.,__'
    base_f1 = base_f1[:layer_size].ljust(layer_size, '.')  # asegurar longitud
    samples = []

    for _ in range(n):
        f2 = ['.'] * layer_size
        pos_w = random.randint(0, layer_size - 5)
        pos_u = min(pos_w + 1 + random.randint(0, 3), layer_size - 1)
        f2[pos_w] = 'w'
        f2[pos_u] = 'u'
        f2 = ''.join(f2)

        f3 = ['_'] * layer_size
        pos_r = random.randint(0, layer_size - 5)
        pos_d = min(pos_r + 1 + random.randint(0, 3), layer_size - 1)
        f3[pos_r] = 'r'
        f3[pos_d] = 'd'
        f3 = ''.join(f3)

        samples.append([base_f1, f2, f3, 'A'])

    return samples

# Paso 3: Generar muestras artificiales
custom_samples = generate_atomicity_samples_rd(10)
separated = data.separate_string_chars(custom_samples)

# Paso 4: Forzar inclusión de todos los caracteres usados en el entrenamiento
caracteres_entrenamiento = list(".,_wurdc")  # He forzado que coincidan con todos los caracteres con el entrenamiento porque si no falla el modelo
layer_size = data.layer_size
extra_samples = []

for c in caracteres_entrenamiento:
    f1 = c * layer_size
    f2 = c * layer_size
    f3 = c * layer_size
    extra_samples.append([f1, f2, f3, 'A'])

separated_extra = data.separate_string_chars(extra_samples)
separated += separated_extra  # Inyectamos muestras de relleno

# Paso 5: One-hot encoding
one_hotted = data.one_hot_encode(separated)

# Paso 6: Eliminar muestras de relleno
one_hotted = one_hotted[:-len(extra_samples)]

# Paso 7: Cargar modelo LSTM entrenado
experiment_name = "interop_1"
best_lstm_model = LSTM_Model(data, 16, 32, 8).to(device)
best_lstm_model.load_state_dict(
    torch.load('./bestmodels/best_LSTM_model_' + experiment_name, map_location=device)
)
best_lstm_model.eval()

# Paso 8: Preparar datos para evaluación
x_test = data.to_lstm_format(one_hotted[:, :-1])
y_test = torch.tensor([0] * len(custom_samples))  # 'A' = índice 0
original_inputs = [s[:3] for s in custom_samples]

# Paso 9: Evaluación
wrong_preds = get_wrong_predictions_bycases(best_lstm_model, x_test, y_test, original_inputs)
print_wrong_preds_bycases(wrong_preds, k=10)


10 Atomicity violation cases wrong predicted:
Sample [',.,_,,_,_,._.,_', '...w.u.........', '_______r___d___'] | Prediction V
Sample [',.,_,,_,_,._.,_', '.........w..u..', '___rd__________'] | Prediction V
Sample [',.,_,,_,_,._.,_', '..........w...u', '________r_d____'] | Prediction V
Sample [',.,_,,_,_,._.,_', 'wu.............', '_r___d_________'] | Prediction V
Sample [',.,_,,_,_,._.,_', '.....w.u.......', '_rd____________'] | Prediction V
Sample [',.,_,,_,_,._.,_', '..........w.u..', '_________r_d___'] | Prediction V
Sample [',.,_,,_,_,._.,_', 'wu.............', '_________r__d__'] | Prediction V
Sample [',.,_,,_,_,._.,_', '.........wu....', '_r___d_________'] | Prediction V
Sample [',.,_,,_,_,._.,_', '..........w.u..', 'rd_____________'] | Prediction V
Sample [',.,_,,_,_,._.,_', '...w...u.......', '___r___d_______'] | Prediction V


0 DeadLock cases wrong predicted:


0 Data race cases wrong predicted:


0 Valid cases wrong predicted:




c:\Users\franm\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:432: UserWarning: X has feature names, but OneHotEncoder was fitted without feature names
  warnings.warn(
c:\Users\franm\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:432: UserWarning: X has feature names, but OneHotEncoder was fitted without feature names
  warnings.warn(
C:\Users\franm\AppData\Local\Temp\ipykernel_9640\3957508601.py:68: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowe

#### Generamos el nuevo dataset y lo evaluamos para el modelo CNN del 4º filtro

In [23]:
import torch
import random
import pandas as pd
from generate_data import Data
from evaluate import get_wrong_predictions_bycases, print_wrong_preds_bycases
from models import CNN_Model
from sklearn.preprocessing import LabelEncoder

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Paso 1: Inicializar generador de datos
data = Data(layer_size=15, interop_distances=[0], permutation_intervals=1)

# Paso 2: Generar muestras con f2='wu' y f3='rd'
def generate_atomicity_samples_rd(n=10):
    layer_size = 15
    base_f1 = ',.,_,,_,_,._.,__'
    base_f1 = base_f1[:layer_size].ljust(layer_size, '.')  # asegurar longitud
    samples = []

    for _ in range(n):
        f2 = ['.'] * layer_size
        pos_w = random.randint(0, layer_size - 5)
        pos_u = min(pos_w + 1 + random.randint(0, 3), layer_size - 1)
        f2[pos_w] = 'w'
        f2[pos_u] = 'u'
        f2 = ''.join(f2)

        f3 = ['_'] * layer_size
        pos_r = random.randint(0, layer_size - 5)
        pos_d = min(pos_r + 1 + random.randint(0, 3), layer_size - 1)
        f3[pos_r] = 'r'
        f3[pos_d] = 'd'
        f3 = ''.join(f3)

        samples.append([base_f1, f2, f3, 'A'])

    return samples

# Paso 3: Generar muestras artificiales
custom_samples = generate_atomicity_samples_rd(10)
separated = data.separate_string_chars(custom_samples)

# Paso 4: Forzar inclusión de todos los caracteres usados en el entrenamiento
caracteres_entrenamiento = list(".,_wurdc")  # He forzado que coincidan con todos los caracteres con el entrenamiento porque si no falla el modelo
layer_size = data.layer_size
extra_samples = []

for c in caracteres_entrenamiento:
    f1 = c * layer_size
    f2 = c * layer_size
    f3 = c * layer_size
    extra_samples.append([f1, f2, f3, 'A'])

separated_extra = data.separate_string_chars(extra_samples)
separated += separated_extra  # Inyectamos muestras de relleno

# Paso 5: One-hot encoding
one_hotted = data.one_hot_encode(separated)

# Paso 6: Eliminar muestras de relleno
one_hotted = one_hotted[:-len(extra_samples)]

# Paso 7: Cargar modelo LSTM entrenado
experiment_name = "interop_1"
best_cnn_model = CNN_Model(data, 32, 64, 4, -1,).to(device)
best_cnn_model.load_state_dict(torch.load('./bestmodels/best_CNN_model_' + experiment_name))
best_cnn_model.eval()

# Paso 8: Preparar datos para evaluación
x_test = data.to_conv_format(one_hotted[:, :-1])
y_test = torch.tensor([0] * len(custom_samples))  # 'A' = índice 0
original_inputs = [s[:3] for s in custom_samples]

# Paso 9: Evaluación
wrong_preds = get_wrong_predictions_bycases(best_cnn_model, x_test, y_test, original_inputs)
print_wrong_preds_bycases(wrong_preds, k=10)


10 Atomicity violation cases wrong predicted:
Sample [',.,_,,_,_,._.,_', '.........wu....', '______r__d_____'] | Prediction R
Sample [',.,_,,_,_,._.,_', '....w...u......', '__rd___________'] | Prediction V
Sample [',.,_,,_,_,._.,_', '........w...u..', '_______rd______'] | Prediction V
Sample [',.,_,,_,_,._.,_', '........w.u....', '_____r_d_______'] | Prediction R
Sample [',.,_,,_,_,._.,_', '.....w...u.....', '___r__d________'] | Prediction R
Sample [',.,_,,_,_,._.,_', 'w..u...........', 'r_d____________'] | Prediction V
Sample [',.,_,,_,_,._.,_', '..........w.u..', 'r__d___________'] | Prediction R
Sample [',.,_,,_,_,._.,_', '......w.u......', '__r_d__________'] | Prediction R
Sample [',.,_,,_,_,._.,_', 'w..u...........', 'r___d__________'] | Prediction R
Sample [',.,_,,_,_,._.,_', '.........wu....', 'rd_____________'] | Prediction V


0 DeadLock cases wrong predicted:


0 Data race cases wrong predicted:


0 Valid cases wrong predicted:




c:\Users\franm\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:432: UserWarning: X has feature names, but OneHotEncoder was fitted without feature names
  warnings.warn(
c:\Users\franm\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:432: UserWarning: X has feature names, but OneHotEncoder was fitted without feature names
  warnings.warn(
C:\Users\franm\AppData\Local\Temp\ipykernel_9640\439274189.py:67: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed

#### Generamos el nuevo dataset y lo evaluamos para el modelo deepset del 4º filtro

In [ ]:
import torch
import random
import pandas as pd
from generate_data import Data
from evaluate import get_wrong_predictions_bycases, print_wrong_preds_bycases
from models import DEEPSET_Model
from sklearn.preprocessing import LabelEncoder

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Paso 1: Inicializar generador de datos con layer_size = 16
# IMPORTANTE: El modelo DEEPSET fue entrenado con layer_size = 16, no 15
# Para que las dimensiones coincidan, debes usar 16 aquí también
data = Data(layer_size=16, interop_distances=[0], permutation_intervals=1)

# Paso 2: Generar muestras con f2='wu' y f3='rd'
def generate_atomicity_samples_rd(n=10):
    layer_size = data.layer_size
    base_f1 = ',.,_,,_,_,._.,__'
    base_f1 = base_f1[:layer_size].ljust(layer_size, '.')  # asegurar longitud
    samples = []

    for _ in range(n):
        f2 = ['.'] * layer_size
        pos_w = random.randint(0, layer_size - 5)
        pos_u = min(pos_w + 1 + random.randint(0, 3), layer_size - 1)
        f2[pos_w] = 'w'
        f2[pos_u] = 'u'
        f2 = ''.join(f2)

        f3 = ['_'] * layer_size
        pos_r = random.randint(0, layer_size - 5)
        pos_d = min(pos_r + 1 + random.randint(0, 3), layer_size - 1)
        f3[pos_r] = 'r'
        f3[pos_d] = 'd'
        f3 = ''.join(f3)

        samples.append([base_f1, f2, f3, 'A'])

    return samples

# Paso 3: Generar muestras artificiales
custom_samples = generate_atomicity_samples_rd(10)
separated = data.separate_string_chars(custom_samples)

# Paso 4: Forzar inclusión de todos los caracteres usados en entrenamiento
caracteres_entrenamiento = list(".,_wurdc")
extra_samples = []

for c in caracteres_entrenamiento:
    f1 = c * data.layer_size
    f2 = c * data.layer_size
    f3 = c * data.layer_size
    extra_samples.append([f1, f2, f3, 'A'])

separated_extra = data.separate_string_chars(extra_samples)
separated += separated_extra

# Paso 5: One-hot encoding
one_hotted = data.one_hot_encode(separated)

# Paso 6: Eliminar muestras de relleno
one_hotted = one_hotted[:-len(extra_samples)]

# Paso 7: Cargar modelo DEEPSET entrenado con 128 de entrada
experiment_name = "interop_1"
best_deepset_model = DEEPSET_Model(data).to(device)
best_deepset_model.load_state_dict(torch.load('./bestmodels/best_DEEPSET_model_' + experiment_name, map_location=device))
best_deepset_model.eval()

# Paso 8: Preparar datos para evaluación
x_test = data.to_conv_format(one_hotted[:, :-1])  # DEEPSET también usa este formato
y_test = torch.tensor([0] * len(custom_samples))  # Clase A = índice 0
original_inputs = [s[:3] for s in custom_samples]

# Paso 9: Evaluación
wrong_preds = get_wrong_predictions_bycases(best_deepset_model, x_test, y_test, original_inputs)
print_wrong_preds_bycases(wrong_preds, k=10)


9 Atomicity violation cases wrong predicted:
Sample [',.,_,,_,_,._.,__', '...........wu...', '___r___d________'] | Prediction V
Sample [',.,_,,_,_,._.,__', '......w..u......', '_______r_d______'] | Prediction V
Sample [',.,_,,_,_,._.,__', '.w...u..........', '________rd______'] | Prediction V
Sample [',.,_,,_,_,._.,__', '.....w..u.......', '_____r___d______'] | Prediction V
Sample [',.,_,,_,_,._.,__', 'w...u...........', '___________r___d'] | Prediction R
Sample [',.,_,,_,_,._.,__', '...w..u.........', '_r_d____________'] | Prediction V
Sample [',.,_,,_,_,._.,__', '.......w.u......', '___r___d________'] | Prediction V
Sample [',.,_,,_,_,._.,__', '........wu......', 'r_d_____________'] | Prediction V
Sample [',.,_,,_,_,._.,__', '.......wu.......', '_______r_d______'] | Prediction V


0 DeadLock cases wrong predicted:


0 Data race cases wrong predicted:


0 Valid cases wrong predicted:




c:\Users\franm\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:432: UserWarning: X has feature names, but OneHotEncoder was fitted without feature names
  warnings.warn(
c:\Users\franm\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:432: UserWarning: X has feature names, but OneHotEncoder was fitted without feature names
  warnings.warn(
C:\Users\franm\AppData\Local\Temp\ipykernel_9640\4114018446.py:69: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowe

#### Generamos el nuevo dataset y lo evaluamos para el modelo deepsetV2 del 4º filtro

In [ ]:
import torch
import random
import pandas as pd
from generate_data import Data
from evaluate import get_wrong_predictions_bycases, print_wrong_preds_bycases
from models import DEEPSETV2_Model
from sklearn.preprocessing import LabelEncoder

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Paso 1: Inicializar generador de datos con layer_size = 16
# IMPORTANTE: El modelo DEEPSETv2 fue entrenado con layer_size = 16, no 15
# Para que las dimensiones coincidan, debes usar 16 aquí también
data = Data(layer_size=16, interop_distances=[0], permutation_intervals=1)

# Paso 2: Generar muestras con f2='wu' y f3='rd'
def generate_atomicity_samples_rd(n=10):
    layer_size = data.layer_size
    base_f1 = ',.,_,,_,_,._.,__'
    base_f1 = base_f1[:layer_size].ljust(layer_size, '.')  # asegurar longitud
    samples = []

    for _ in range(n):
        f2 = ['.'] * layer_size
        pos_w = random.randint(0, layer_size - 5)
        pos_u = min(pos_w + 1 + random.randint(0, 3), layer_size - 1)
        f2[pos_w] = 'w'
        f2[pos_u] = 'u'
        f2 = ''.join(f2)

        f3 = ['_'] * layer_size
        pos_r = random.randint(0, layer_size - 5)
        pos_d = min(pos_r + 1 + random.randint(0, 3), layer_size - 1)
        f3[pos_r] = 'r'
        f3[pos_d] = 'd'
        f3 = ''.join(f3)

        samples.append([base_f1, f2, f3, 'A'])

    return samples

# Paso 3: Generar muestras artificiales
custom_samples = generate_atomicity_samples_rd(10)
separated = data.separate_string_chars(custom_samples)

# Paso 4: Forzar inclusión de todos los caracteres usados en entrenamiento
caracteres_entrenamiento = list(".,_wurdc")
extra_samples = []

for c in caracteres_entrenamiento:
    f1 = c * data.layer_size
    f2 = c * data.layer_size
    f3 = c * data.layer_size
    extra_samples.append([f1, f2, f3, 'A'])

separated_extra = data.separate_string_chars(extra_samples)
separated += separated_extra

# Paso 5: One-hot encoding
one_hotted = data.one_hot_encode(separated)

# Paso 6: Eliminar muestras de relleno
one_hotted = one_hotted[:-len(extra_samples)]

# Paso 7: Cargar modelo DEEPSET entrenado con 128 de entrada
experiment_name = "interop_1"
best_deepsetv2_model = DEEPSETV2_Model(data).to(device)
best_deepsetv2_model.load_state_dict(torch.load('./bestmodels/best_DEEPSETV2_model_' + experiment_name))
best_deepsetv2_model.eval()

# Paso 8: Preparar datos para evaluación
x_test = data.to_conv_format(one_hotted[:, :-1])  # DEEPSET también usa este formato
y_test = torch.tensor([0] * len(custom_samples))  # Clase A = índice 0
original_inputs = [s[:3] for s in custom_samples]

# Paso 9: Evaluación
wrong_preds = get_wrong_predictions_bycases(best_deepsetv2_model, x_test, y_test, original_inputs)
print_wrong_preds_bycases(wrong_preds, k=10)


10 Atomicity violation cases wrong predicted:
Sample [',.,_,,_,_,._.,__', '......w..u......', '____r__d________'] | Prediction V
Sample [',.,_,,_,_,._.,__', '.........w...u..', '_____r_d________'] | Prediction V
Sample [',.,_,,_,_,._.,__', '...w..u.........', '___________r__d_'] | Prediction V
Sample [',.,_,,_,_,._.,__', '.......w.u......', '____r__d________'] | Prediction V
Sample [',.,_,,_,_,._.,__', '.........w..u...', 'r__d____________'] | Prediction V
Sample [',.,_,,_,_,._.,__', 'w...u...........', '_____rd_________'] | Prediction V
Sample [',.,_,,_,_,._.,__', '........w.u.....', 'r_d_____________'] | Prediction V
Sample [',.,_,,_,_,._.,__', '.....w...u......', '________rd______'] | Prediction V
Sample [',.,_,,_,_,._.,__', '........wu......', '________rd______'] | Prediction V
Sample [',.,_,,_,_,._.,__', 'w..u............', '________r_d_____'] | Prediction V


0 DeadLock cases wrong predicted:


0 Data race cases wrong predicted:


0 Valid cases wrong predicted:




c:\Users\franm\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:432: UserWarning: X has feature names, but OneHotEncoder was fitted without feature names
  warnings.warn(
c:\Users\franm\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:432: UserWarning: X has feature names, but OneHotEncoder was fitted without feature names
  warnings.warn(
C:\Users\franm\AppData\Local\Temp\ipykernel_9640\4099876642.py:68: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowe

#### Generamos el nuevo dataset y lo evaluamos para el modelo feedforward del 4º filtro

In [25]:
import torch
import random
import pandas as pd
from generate_data import Data
from evaluate import get_wrong_predictions_bycases, print_wrong_preds_bycases
from models import FEEDFORWARD_Model
from sklearn.preprocessing import LabelEncoder

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Paso 1: Inicializar generador de datos con layer_size = 16
# IMPORTANTE: El modelo DEEPSETv2 fue entrenado con layer_size = 16, no 15
# Para que las dimensiones coincidan, debes usar 16 aquí también
data = Data(layer_size=16, interop_distances=[0], permutation_intervals=1)

# Paso 2: Generar muestras con f2='wu' y f3='rd'
def generate_atomicity_samples_rd(n=10):
    layer_size = data.layer_size
    base_f1 = ',.,_,,_,_,._.,__'
    base_f1 = base_f1[:layer_size].ljust(layer_size, '.')  # asegurar longitud
    samples = []

    for _ in range(n):
        f2 = ['.'] * layer_size
        pos_w = random.randint(0, layer_size - 5)
        pos_u = min(pos_w + 1 + random.randint(0, 3), layer_size - 1)
        f2[pos_w] = 'w'
        f2[pos_u] = 'u'
        f2 = ''.join(f2)

        f3 = ['_'] * layer_size
        pos_r = random.randint(0, layer_size - 5)
        pos_d = min(pos_r + 1 + random.randint(0, 3), layer_size - 1)
        f3[pos_r] = 'r'
        f3[pos_d] = 'd'
        f3 = ''.join(f3)

        samples.append([base_f1, f2, f3, 'A'])

    return samples

# Paso 3: Generar muestras artificiales
custom_samples = generate_atomicity_samples_rd(10)
separated = data.separate_string_chars(custom_samples)

# Paso 4: Forzar inclusión de todos los caracteres usados en entrenamiento
caracteres_entrenamiento = list(".,_wurdc")
extra_samples = []

for c in caracteres_entrenamiento:
    f1 = c * data.layer_size
    f2 = c * data.layer_size
    f3 = c * data.layer_size
    extra_samples.append([f1, f2, f3, 'A'])

separated_extra = data.separate_string_chars(extra_samples)
separated += separated_extra

# Paso 5: One-hot encoding
one_hotted = data.one_hot_encode(separated)

# Paso 6: Eliminar muestras de relleno
one_hotted = one_hotted[:-len(extra_samples)]

# Paso 7: Cargar modelo DEEPSET entrenado con 128 de entrada
experiment_name = "interop_1"
best_feedforward_model = FEEDFORWARD_Model(data).to(device)
best_feedforward_model.load_state_dict(torch.load('./bestmodels/best_FEEDFORWARD_model_' + experiment_name))
best_feedforward_model.eval()

# Paso 8: Preparar datos para evaluación
x_test = data.to_conv_format(one_hotted[:, :-1])  # DEEPSET también usa este formato
y_test = torch.tensor([0] * len(custom_samples))  # Clase A = índice 0
original_inputs = [s[:3] for s in custom_samples]

# Paso 9: Evaluación
wrong_preds = get_wrong_predictions_bycases(best_feedforward_model, x_test, y_test, original_inputs)
print_wrong_preds_bycases(wrong_preds, k=10)


8 Atomicity violation cases wrong predicted:
Sample [',.,_,,_,_,._.,__', '.w..u...........', '_______r_d______'] | Prediction V
Sample [',.,_,,_,_,._.,__', '....w.u.........', '__________r___d_'] | Prediction V
Sample [',.,_,,_,_,._.,__', '.......wu.......', '_________r___d__'] | Prediction V
Sample [',.,_,,_,_,._.,__', 'w...u...........', '_r_d____________'] | Prediction V
Sample [',.,_,,_,_,._.,__', '.........w..u...', '________rd______'] | Prediction V
Sample [',.,_,,_,_,._.,__', '......w.u.......', 'r_d_____________'] | Prediction V
Sample [',.,_,,_,_,._.,__', '.w.u............', '_r_d____________'] | Prediction V
Sample [',.,_,,_,_,._.,__', '.....w...u......', '__________rd____'] | Prediction V


0 DeadLock cases wrong predicted:


0 Data race cases wrong predicted:


0 Valid cases wrong predicted:




c:\Users\franm\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:432: UserWarning: X has feature names, but OneHotEncoder was fitted without feature names
  warnings.warn(
c:\Users\franm\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:432: UserWarning: X has feature names, but OneHotEncoder was fitted without feature names
  warnings.warn(
C:\Users\franm\AppData\Local\Temp\ipykernel_9640\2299266133.py:68: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowe